In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sympy import sieve
import time
import tensorflow as tf
from itertools import combinations,permutations,product
from sklearn.neighbors import KDTree
from common import *
from os import system

In [2]:
def two_way_distance(nodes,solution,prime,onlyten=False):
    global pen
    s = nodes[solution[:-1]]
    e = nodes[solution[1:]]
    raw_d = distance(s,e)
    rever_raw_d = raw_d[::-1]
    
    n = len(raw_d)
    dist = np.zeros((20,n),dtype=float)
    zero_column = np.zeros((20,1),dtype=float)
    
    notprime = np.logical_not(np.isin(solution,prime))
    notprime_forward = notprime[:-1]
    notprime_backward = notprime[-1:0:-1]
    
    if not onlyten:
        for offset in range(10):
            dist[offset,9-offset::10] += raw_d[9-offset::10] * notprime_forward [9-offset::10] * pen
            dist[offset+10,9-offset::10] += (rever_raw_d[9-offset::10] * notprime_backward[9-offset::10] * pen)
            dist[offset+10] = dist[offset+10][::-1]
    else:
        for offset in range(10):
            dist[offset,9-offset::10] += raw_d[9-offset::10] * pen
            dist[offset+10,9-offset::10] += (rever_raw_d[9-offset::10] * pen)
    dist = np.concatenate([zero_column,dist],axis=-1)
    return dist.cumsum(axis=-1)

def excute_swap(code,links,solution):
    permu = permus[code//num_prod]
    prod = prods[code%num_prod]
    segs = links[1:-1].reshape(opt-1,2).copy()
    segs[:,1] += 1
    
    copies = []
    for i in permu:
        a,b = segs[i]
        copies.append(solution[a:b].copy())
    
    solution[segs[0,0]:segs[-1,-1]] = np.concatenate([copies[i][::prod[i]] for i in range(len(segs))])
    
    
def generate(opt):
    from itertools import combinations,permutations,product
    links = np.arange(opt*2)
    r = opt-1
    first,last = links[[0,-1]]
    pair = links[1:-1].reshape(r,2)
    
    pair_idx = list(range(r))
    pair_combo = []
    for _ in range(opt):
        pair_combo += list(combinations(pair_idx, _))
        
    num_possible_idx = len(pair_combo)
    possible_idx_from_pair = np.zeros((num_possible_idx,r),dtype=int)
    possible_idx_from_pos = np.zeros((num_possible_idx,),dtype=int)
    
    for i,pre in enumerate(pair_combo):
        pre_pairs_idx = list(pre)
        possible_idx_from_pair[i,pre_pairs_idx] = 1
        possible_idx_from_pos[i] = len(pre_pairs_idx)
    
    
    
    permus = np.array(list(permutations(np.arange(r))))
    prods = np.array(list(product([1,-1],repeat = r)))
    
    permus_n = len(permus)
    prods_n = len(prods)
    
    mask_p = np.zeros((permus_n*prods_n,opt),dtype=int)
    mask_p[:,0] = 0
    
    which_connect = np.zeros((opt,permus_n*prods_n,2),dtype=int)
    which_mask_t = np.zeros((opt,permus_n*prods_n),dtype=int)
    
    mask_t_node = np.zeros((permus_n*prods_n,opt-2,r),dtype=int)
    mask_t_pos = np.zeros((permus_n*prods_n,opt-2),dtype=int)
    
    which_new_pos = np.zeros((r, permus_n),dtype=int)
    
    code_border = np.zeros((permus_n*prods_n, 2),dtype=int)
    
    for i in range(opt-2):
        mask_t_pos[:,i] = i + 1
        
    offset_idx_from_node = np.zeros((r,permus_n*2,r),dtype=int)
    offset_idx_from_pos = np.zeros((r,permus_n*2),dtype=int)
    which_middle_idx = np.zeros((r,permus_n*prods_n),dtype=int)
    reconstruct = []
    for i,permu in enumerate(permus):
        
        middle = pair[[permu]]
        where = np.argsort(permu)
        
        for j,pro in enumerate(prods):
            
               
            middle_ = middle.copy()
            for k,flip in enumerate(pro):
                
                middle_[k] = middle_[k][::flip]
                mask_p[i*prods_n+j,k+1] = middle_[k][-1]
                
                actual_seg = permu[k]
                before_current = permu[:k]
                if k>0:
                    mask_t_node[i*prods_n+j,k-1,before_current] = 1
                if flip == 1:
                    which_middle_idx[actual_seg,i*prods_n+j] = i
                else:
                    which_middle_idx[actual_seg,i*prods_n+j] = i+permus_n
            
            row = np.concatenate([[first],middle_.ravel(),[last]])
            for w in range(opt):
                rr,cc = row[w*2:w*2+2]
                which_connect[w,i*prods_n+j] = [rr,cc]
            reconstruct.append(row)
            
            nochange = row == links
            for _,noc in enumerate(nochange):
                if not noc:
                    code_border[i*prods_n+j,0] = _ - 1
                    break
            for _,noc in enumerate(nochange[::-1]):
                if not noc:
                    code_border[i*prods_n+j,1] = len(nochange) - _
                    break
                    
        for k in range(r):
            actual_seg = permu[k]
            before_current = permu[:k]

            offset_idx_from_pos[actual_seg,i] = k+1
            offset_idx_from_pos[actual_seg,i+permus_n] = k+1

            offset_idx_from_node[actual_seg,i,[before_current]] = 1
            offset_idx_from_node[actual_seg,i+permus_n,[before_current]] = 1
        
    
        
        for k in range(opt):
            if k<opt-1:
                actual_seg = permu[k]
                before_current = np.sort(permu[:k]).tolist()
                before_current = tuple(before_current)
                which_combo = pair_combo.index(before_current)
                
                which_mask_t[k,i*prods_n:(i+1)*prods_n] = which_combo
                which_new_pos[actual_seg, i] = which_combo
            else:
                which_mask_t[k,i*prods_n:(i+1)*prods_n] = len(pair_combo)-1
    reconstruct = np.array(reconstruct)      
    
    return reconstruct,permus,prods,which_middle_idx,offset_idx_from_node,\
           offset_idx_from_pos,mask_p,mask_t_pos,mask_t_node,which_connect,\
           possible_idx_from_pair,possible_idx_from_pos,which_mask_t,which_new_pos,code_border

In [3]:
p = problem(base_path='./solution/base5.csv')
print(p.score(p.base,raw=False))
pen = 0.1
solution = p.base.copy()
nodes = p.nodes.copy()
prime = p.prime.copy()
prime_set = p.prime_set
n = len(nodes)

dist = two_way_distance(nodes,solution,prime)
notprime = np.isin(np.arange(n),prime,invert=True)

1516273.2322748974


In [4]:
import collections 
import bisect


def weighted_interval_scheduling(good_border,good_imp):

    S = collections.defaultdict(int)
    
    start = good_border[:,0]
    end = good_border[:,1]

    S[0] = 0
    S[1] = good_imp[0]

    for i in range(2, len(good_border)+1):
        #S[i] = max(S[i-1], G[i-1][2] + S[ bisect.bisect_left(end, start[i-1]) ])
        S[i] = max(S[i-1], good_imp[i-1] + S[ np.searchsorted(end, start[i-1], side='left') ])
    
    who = []
    j = len(good_imp)
    while j>0:
        pj = np.searchsorted(end, start[j-1], side='left')
        jin = good_imp[j-1] + S[pj]
        jnotin = S[j-1]
        if jin >= jnotin:
            who.append(j-1)
            j = pj
        else:
            j-=1
    return S[len(good_imp)], who

In [7]:
def reset_graph(opt):
    global pen,num_permu,num_prod,permus,prods,code_border
    reconstruct,permus,prods,which_middle_idx,\
    offset_idx_from_node,offset_idx_from_pos,\
    mask_p,mask_t_pos,mask_t_node,which_connect,\
    possible_idx_from_pair,possible_idx_from_pos,\
    which_mask_t,which_new_pos, code_border = generate(opt)

    num_permu = len(permus)
    num_prod = len(prods)
    num_reconstruct = len(reconstruct)
    
    global sess,dist,solution
    try:
        sess.close()
    except:
        pass
    
    tf.reset_default_graph()
    #constant
    _nodes = tf.constant(nodes, dtype='float64', shape=(n,2))
    _notprime = tf.constant(notprime, dtype='bool', shape=(n,)) 
    #_neighbors 

    _reconstruct = tf.constant(reconstruct.T, dtype='int32') 
    #permus
    #prods
    _which_connect = tf.constant(which_connect, dtype='int32')
    _which_middle_idx = tf.constant(which_middle_idx, dtype='int32')
    _offset_idx_from_node = tf.constant(offset_idx_from_node[:,:num_permu,:,np.newaxis], dtype='int32') 
    _offset_idx_from_pos = tf.constant(offset_idx_from_pos[:,:num_permu,np.newaxis], dtype='int32') 
    _mask_p = tf.constant(mask_p.T, dtype='int32') 
    _possible_idx_from_pair = tf.constant(possible_idx_from_pair[:,:,None], dtype='int32') 
    _possible_idx_from_pos = tf.constant(possible_idx_from_pos[:,None], dtype='int32') 
    _which_mask_t = tf.constant(which_mask_t, dtype='int32') 
    _which_new_pos = tf.constant(which_new_pos, dtype='int32') 

    #bookkeepers
    _solution = tf.Variable(solution, dtype='int32')
    _dist = tf.Variable(dist.T, dtype='float64')

    #place holder
    _links = tf.placeholder(dtype='int32', shape=(opt*2,None),name='links')
    _link = tf.placeholder(dtype='int32', shape=(opt*2,))
    _code = tf.placeholder(dtype='int32')

    #wip
    _linksnode = tf.gather(_solution, _links)
    _linksnodecoor = tf.gather(_nodes, _linksnode)
    _pairstep = _links[2:-1:2] - _links[1:-1:2]
    _linksprime = tf.gather(_notprime, _linksnode)

    _coor1 = tf.stack([_linksnodecoor]*(opt*2),axis=0)
    _coor2 = tf.stack([_linksnodecoor]*(opt*2),axis=1)
    _connectorrawdist = (_coor1 - _coor2) ** 2
    _connectorrawdist = tf.sqrt(tf.reduce_sum(_connectorrawdist, axis=-1))
    _connectorrawdist = tf.gather_nd(_connectorrawdist, _which_connect)

    _maskprime = tf.gather(_linksprime, _mask_p)

    _tenpos = tf.reduce_sum(_possible_idx_from_pair*_pairstep , axis=1)\
                   + _possible_idx_from_pos\
                   + _links[0] + 1
    _masktenpos = tf.equal(_tenpos%10, 0)
    _maskten = tf.gather(_masktenpos, _which_mask_t)

    _mask = tf.logical_and(_maskprime, _maskten)
    
    _connectorpenalty = tf.where(_mask, _connectorrawdist*pen, tf.zeros_like(_connectorrawdist, dtype='float64'))
    _connectorpenalty = tf.reduce_sum(_connectorpenalty, axis=0)
    
    _connectorpenaltyonlyten = tf.where(_maskten, _connectorrawdist*pen, tf.zeros_like(_connectorrawdist, dtype='float64'))
    _connectorpenaltyonlyten = tf.reduce_sum(_connectorpenaltyonlyten, axis=0)
    
    _tsp_raw = tf.reduce_sum(_connectorrawdist, axis=0)
    _tsp = _tsp_raw - _tsp_raw[0:1]
    _tsploss = _tsp / _tsp_raw[0:1]
    
    ##segment
    _forwardat = _links[1:-1:2,None,:]
    _backwardat = len(solution)-1 - _links[2:-1:2,None,:]

    _possibleforoffset = (_tenpos-_forwardat)%10
    _possiblebackoffset = (_tenpos-_backwardat)%10 + 10
    _forwardoffset = tf.batch_gather(_possibleforoffset, _which_new_pos)
    _backwardatoffset = tf.batch_gather(_possiblebackoffset,_which_new_pos)

    _whichoffset = tf.concat([_forwardoffset,_backwardatoffset], axis=1)

    _segpenalty = tf.gather(_dist,_links[2:-1:2]) - tf.gather(_dist,_links[1:-1:2])
   
    _legpenalty = tf.batch_gather(_segpenalty, tf.transpose(_whichoffset,[0,2,1] ))
    _legpenalty = tf.transpose(_legpenalty,[0,2,1])
    _legpenalty = tf.batch_gather(_legpenalty, _which_middle_idx)
    _segments = tf.reduce_sum(_legpenalty, axis=0)


    #final
    _penalty = _connectorpenalty + _segments
    _penalty = tf.subtract(_penalty, _penalty[0:1], name='out')
    
    _penaltyonlyten = _connectorpenaltyonlyten + _segments
    _penaltyonlyten = tf.subtract(_penaltyonlyten, _penaltyonlyten[0:1], name='out')
    
    _out = _penalty + _tsp
    _out2 = _penalty + _tsp*2
    _out3 = _penalty + _tsp*3
    _out4 = _penalty + _tsp*4
    
    
    _outonlyten = _penaltyonlyten + _tsp
    
    #updates
    _newsolution = tf.placeholder(dtype='int32', name='newsolution' )
    _newdist = tf.placeholder( dtype='float64', name='newdist' )

    _updatesolution = tf.assign(_solution, _newsolution, name='updatesolution')
    _updatedist = tf.assign(_dist, _newdist, name='updatedist')
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    return _links,_out,_newsolution,_newdist,_tsp

In [97]:
def score_print(solution):
    print(p.score(solution),p.score(solution,raw=True),np.isin(solution[9::10],prime).sum())

def recalc_bk(solution,nodes,prime):
    whereat = np.argsort(solution[:-1])
    dist = two_way_distance(nodes,solution,prime)
    return whereat,dist

def generate_links(node,kdt,whereat,num_neighbor,opt):
    candidate = kdt.query(nodes[[node]], num_neighbor, return_distance =False)[0]
    pos = whereat[candidate]
    links = np.array(list(combinations(pos,opt)))
    links = np.sort(links,axis=1).T
    links = np.repeat(links,2,axis=0)
    links[1::2] += 1
    return links

def reset_goods():
    return [],[],[]

def do_swaps(good_link,good_code,good_imp,solution,many=False,check=False):
    if len(good_link)==0:
        return 0,0
    good_link = np.concatenate(good_link,axis=0)
    good_code = np.concatenate(good_code,axis=0)
    good_imp = np.concatenate(good_imp,axis=0)

    border_idx = code_border[good_code]
    row = np.arange(len(good_link))[:,None]
    row = np.tile(row,[1,2])
    good_border = good_link[row, border_idx]

    order = np.argsort(good_border[:,1])

    good_link = good_link[order]
    good_code = good_code[order]
    good_imp = -good_imp[order]
    if many:
        good_imp = np.ones_like(good_imp)
    good_border = good_border[order]

    gain,chosen = weighted_interval_scheduling(good_border,good_imp)
    if check:
        return good_link[chosen],good_code[chosen],good_border[chosen],good_imp[chosen]
    for i in chosen:
        link = good_link[i]
        code = good_code[i]

        excute_swap(code,link,solution)
    return gain,len(chosen)

def do(solution,whereat,dist,nodes_to_check,_target, num_neighbor,opt,many=False):
    
    good_link,good_code,good_imp = reset_goods()

    a=time.time()
    for i,node in enumerate(nodes_to_check):
        links = generate_links(node,kdt,whereat,num_neighbor,opt)
        batch = links.shape[1]
        if batch>0:
            target = sess.run(_target,feed_dict={_links:links})
            code,which_link = np.where(target<0)

            if len(code)>0:
                good_link.append(links[:,which_link].T)
                good_code.append(code)
                good_imp.append(target[code,which_link])

    #print(time.time()-a)
    gain,amount = do_swaps(good_link,good_code,good_imp,solution,many)
    print(gain,amount)
    whereat,dist = recalc_bk(solution,nodes,prime)
    sess.run(['updatesolution','updatedist'],feed_dict={_newsolution:solution,
                                                        _newdist:dist.T})
    score_print(solution)
    return gain

def do_worse(solution,whereat,dist,nodes_to_check,_target, num_neighbor,opt,many=True):
    
    good_link,good_code,good_imp = reset_goods()

    a=time.time()
    print('---------')
    print(len(nodes_to_check))
    for i,node in enumerate(nodes_to_check):
        if i%10000==0:
            print(i,len(good_link))

        links = generate_links(node,kdt,whereat,num_neighbor,opt)
        batch = links.shape[1]
        if batch>0:
            target = sess.run(_target,feed_dict={_links:links})
            code,which_link = np.where(target<0)

            if len(code)>0:
                good_link.append(links[:,which_link].T)
                good_code.append(code)
                good_imp.append(target[code,which_link])

    print(time.time()-a)
    print(len(good_link))

    great_link,great_code,great_border,great_imp = do_swaps(good_link,good_code,good_imp,solution,many,True)
    return great_link,great_code,great_border,great_imp

    print(gain,amount)
    whereat,dist = recalc_bk(solution,nodes,prime)
    sess.run(['updatesolution','updatedist'],feed_dict={_newsolution:solution,
                                                        _newdist:dist.T})
    score_print(solution)
    return gain

In [6]:
def write_tsp_file(fname, matrix, name):
    with open(fname, 'w') as fp:
        fp.write("NAME: {}\n".format(name))
        fp.write("TYPE: ATSP\n")
        fp.write("DIMENSION: {}\n".format(len(matrix)))
        fp.write("EDGE_WEIGHT_TYPE: EXPLICIT\n")
        fp.write("EDGE_WEIGHT_FORMAT: FULL_MATRIX\n")
        fp.write("EDGE_WEIGHT_SECTION\n")
        for _,row in enumerate(matrix):
            for d in row:
                fp.write("{} ".format(int(d*1000)))
            fp.write("\n")
        #fp.write("-1\n")
        fp.write("EOF\n")

def read_sub_result():
    with open('after_tour.txt','r') as f:
        line = 1
        content = '123'
        sol = []
        while content != '':
            content = f.readline()
            if line >=7:

                sol.append(content)
            line+=1


    sol = sol[:-2]
    sol = [int(line.split('\n')[0]) for line in sol]
    sol[-1] = 1
    sol = np.array(sol)
    sol-=1
    return sol

def write_initial_tour(sub_tour):#make .tsp problem
    fname =  'tour.txt'
    with open(fname, 'w') as f:
        f.write('NAME : santa\n')
        f.write('COMMENT : sub_initial\n')
        f.write('TYPE : ATSP\n')
        f.write('DIMENSION : %d\n' % (len(sub_tour)-1))
        f.write('TOUR_SECTION\n')
        for node in sub_tour[:-1]:
            f.write('%s\n' % int(node+1))
        f.write('-1\n')
        f.write('EOF')

In [62]:
def generate_box(x,y,l,nodes=nodes):
    box = np.array([[x,x+l],[y,y+l]])
    inthebox = np.where((nodes[:,0]>=box[0,0]) & (nodes[:,0]<=box[0,1]) & (nodes[:,1]>=box[1,0]) & (nodes[:,1]<=box[1,1]))[0]
    return inthebox

x,y,l = 900,2600,180
inthebox = generate_box(x,y,l)
print(len(inthebox))

475


In [118]:
p = problem(base_path='./solution/submit_base5_7.csv')
solution = p.base.copy()
score_print(solution)

kdt = KDTree(nodes)

for qwe in range(2):
    print(qwe)
    print('********************************')
    ##############################
    opt = 3
    whereat,dist = recalc_bk(solution,nodes,prime)
    _links,_out,_newsolution,_newdist,_tsp = reset_graph(opt)

    great_link,great_code,great_border,great_imp = do_worse(solution,whereat,dist,solution[:-1],_tsp, 25,opt,False)
    print(len(great_link))

    score_print(solution)


    for i in range(len(great_link)):
        print('-------')
        opt = 3
        whereat,dist = recalc_bk(solution,nodes,prime)
        _links,_out,_newsolution,_newdist,_tsp = reset_graph(opt)

        link = great_link[i]
        code = great_code[i]
        old_solution = solution.copy()
        old_score = p.score(old_solution)

        b_from = great_border[i,0]
        b_to = great_border[i,1]+1
        print(b_from,b_to)

        excute_swap(code,link,solution)
        score_print(solution)


        ############################

        opt = 4
        whereat,dist = recalc_bk(solution,nodes,prime)
        _links,_out,_newsolution,_newdist,_tsp = reset_graph(opt)


        inthebox = solution[max(0,b_from-10):min(n,b_to+10)]
        do(solution,whereat,dist,inthebox,_out, 30,opt)
        #score_print(solution)
        new_score = p.score(solution)
        if new_score>old_score:
            solution = old_solution


1515157.668385371 1503283.798441696 3658
0
********************************


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:93: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


---------
197769
0 0
10000 393
20000 704
30000 1032
40000 1349
50000 1815
60000 2241
70000 2471
80000 2867
90000 3173
100000 3576
110000 4010
120000 4412
130000 4890
140000 5242
150000 5804
160000 6210
170000 6584
180000 6955
190000 7365
608.4284110069275
7782
435
1515157.668385371 1503283.798441696 3658
-------
197662 197750
1515159.9728630052 1503283.0648400558 3656
2.3044776342793263 1
1515157.668385371 1503283.798441696 3658
-------
197256 197281
1515158.2476588208 1503283.1433348805 3657
0.5792734497024519 1
1515157.668385371 1503283.798441696 3658
-------
196646 196687
1515157.9274778203 1503283.3971439302 3657
0.29650791542250143 1
1515157.6309699048 1503285.6499129422 3660
-------
196480 196494
1515158.1602338864 1503284.801687809 3659
0.529263981592436 1
1515157.6309699048 1503285.6499129422 3660
-------
195975 196202
1515158.6512933285 1503285.6057891524 3659
1.0203234237422585 1
1515157.6309699048 1503285.6499129422 3660
-------
195870 195882
1515158.3442680177 1503285.34388

KeyboardInterrupt: 

In [122]:
p = problem(base_path='./solution/submit_base5_8.csv')
solution = p.base.copy()
score_print(solution)

kdt = KDTree(nodes)

for seg_span in [1000,2000,4000]:
    print('#############################')
    print('span: %s'%seg_span)
    for _ in range(80,n,int(seg_span*0.8)):
        print('-------------------')
        print(_)
        inthebox = solution[_:_+seg_span]

        opt = 3
        whereat,dist = recalc_bk(solution,nodes,prime)
        _links,_out,_newsolution,_newdist,_tsp = reset_graph(opt)
        
        old_solution = solution.copy()
        old_score = p.score(old_solution)
        worse = do(solution,whereat,dist,inthebox,_tsp, 30,opt,True)

        ####################################
        ####################################
        if worse>0:
            opt = 4
            whereat,dist = recalc_bk(solution,nodes,prime)
            _links,_out,_newsolution,_newdist,_tsp = reset_graph(opt)

            do(solution,whereat,dist,inthebox,_out, 30,opt)
            
            new_score = p.score(solution)
            if new_score>old_score:
                solution = old_solution

1515145.5041604044 1503288.639245481 3679
#############################
span: 1000
-------------------
80


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:93: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


1.0 1
1515146.2019346282 1503287.8066811739 3677
0.6977742236807067 1
1515145.5041604044 1503288.639245481 3679
-------------------
880
3.0 3
1515147.9888999937 1503287.6429009233 3675
2.484739589279452 3
1515145.5041604044 1503288.639245481 3679
-------------------
1680
3.0 3
1515147.7676191095 1503286.883939221 3675
2.2634587050439707 3
1515145.5041604044 1503288.639245481 3679
-------------------
2480
4.0 4
1515148.3263477192 1503286.445172094 3676
2.822187314799573 4
1515145.5041604044 1503288.639245481 3679
-------------------
3280
8.0 8
1515155.4668798242 1503285.6168871857 3668
9.962719419760656 8
1515145.5041604044 1503288.639245481 3679
-------------------
4080
5.0 5
1515152.6192234494 1503286.1374433006 3669
7.115063044983264 5
1515145.5041604044 1503288.639245481 3679
-------------------
4880
4.0 4
1515147.280807909 1503287.3741510015 3677
1.77664750456744 4
1515145.5041604044 1503288.639245481 3679
-------------------
5680
4.0 4
1515151.4952241238 1503287.1568161526 3672
5.

KeyboardInterrupt: 

In [123]:
#write result
pd.DataFrame({'Path':solution}).to_csv('./solution/submit_base5_9.csv',index=False)

In [7]:
import pickle
with open('candidates.txt','rb') as f:
    candidates = pickle.load(f)
    
p = problem(base_path='./solution/submit_base5_6.csv')
solution = p.base.copy()
whereat = np.argsort(solution[:-1])

for i in range(1,n):
    already = candidates[i]
    at = whereat[i]
    left = solution[at-1]
    right = solution[at+1]
    if left not in already:
        candidates[i].append(left)
    if right not in already:
        candidates[i].append(right)

startt = time.time()
fours = set()
for i in range(n):
    for j in candidates[i]:
        for k in candidates[j]:
            for p in candidates[k]:
                links = sorted((i,j,k,p))
                if j>i and k>j and p>k:
                    fours.add(tuple(links))
print(len(fours))
print(time.time()-startt)
fours = list(fours)
fours = np.array(fours)

startt = time.time()
threes = set()
for i in range(n):
    for j in candidates[i]:
        for k in candidates[j]:
            links = sorted((i,j,k))
            if j>i and k>j:
                threes.add(tuple(links))
print(len(threes))
print(time.time()-startt)
threes = list(threes)
threes = np.array(threes)

startt = time.time()
twos = set()
for i in range(n):
    for j in candidates[i]:
        #for k in candidates[j]:
        links = sorted((i,j))
        if j>i:
            twos.add(tuple(links))
print(len(twos))
print(time.time()-startt)
twos = list(twos)
twos = np.array(twos)